tldr - This data has been downloaded from my "saved jobs" on LinkedIn. This notebook helps process and sort the data into our groups for the experiment.

# Summary
I'm running an experiment on how well two different resume's perform. Does a professionally written resume perform better than one I've written myself?			

# Hypothesis
$H_{0} = \text{The resume and cover letter provided by Haley Stock perform the same as my own self-written resume and cover letter.}$

$H_{a} = \text{The professionally-written resume and cover letter have a different interview invitation rate than self-written resume and cover letter.}$

| Name                       | Definition                                                                                                   | Example Format | P1   | P2 (from detectable difference below) |
|----------------------------|-------------------------------------------------------------------------------------------------------------|----------------|------|----------------------------------------|
| Interview Invitation Rate  | This is the percent of applications that receive an interview invite. It shows how interested companies are in my resume. | 0.50%         | 8%   | 13%                                    |
| Application Response Rate  | This is the percent of applications that receive a rejection/interview invite within 48 hrs. It shows how confident companies are in my qualifications. | 1%             |      |                                        |


# Data Collection Process
**LinkedIn:** I searched for "data" and remote in US roles. I then saved the roles that met the criteria below to a csv (via LinkedIn's data request form).

# Inclusion Criteria
Here are the kinds of jobs I'm interested in. I'm going to include these roles in the experiment based on the logic listed below.

| **Category**            | **Details**                                                                                                                                                          |
|:------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Location**            | Remote work only                                                                                                                                                     |
| **Minimum Job Roles**   | Data Scientist, Senior Data/Product Analyst, or Manager/Lead<br>No Applied Scientist, Decision Science, Analytics Engineer, or Researcher roles (significantly different than my experience would allow)<br>No Product Manager roles (not interested in product management)<br>No Director level roles (largely above my level) <br>Minimum salary of $100k/yr|
| **Type**                | Full time and contracted                                                                                                                                             |
| **Duplicate Job Roles** | Sometimes companies post multiple versions of a job (by location or reposts after a few weeks have gone by). Will do my best to dedupe these in the final "saved jobs" list. |


# Potential Pitfalls and Adjustments
1) *Cutting sample collection short.* **Discussion:** See power analysis below. Peer review pending.
2) *Confirming random assignment.* **Discussion:** We will randomizing at cluster level (repeat/not yet applied company + Analyst/Scientist/or Manager role).
3) *Random assignment fails to equally distribute "heavy" segments across A/B.* **Discussion:** Equally distribute the following segments and assess post-assignment:
   - Recency of job posting
   - "Repeat" companies (see "8)" below)
   - Role type: Analyst, Scientist, or Manager
4) *Ensure the Ceteris Parabus assumption* (that is, ensure all else is same between treatment/control besides the treatment). **Discussion:** Use the exact resume and cover letter provided for both.
5) *Cross-contamination for treatment and control groups.* **Discussion:** This could be an issue if different role types have the same recruiter.
6) *Multiple comparisons can lead to higher false positive rate.* **Discussion:** Only making one comparison -- professionally written resume vs. self-written resume.
7) *Simpson's paradox due to graduated roll-outs.* **Discussion:** I'm only performing one testing period, so this point is moot. Could simply look at final period if so.
8) *Primacy or novelty effect.* **Discussion:** This could be a problem, if I previously applied for a role with a company before conducting the experiment. I can adjust for this by running the experiment longer, but that's not really helpful given my current situation. Instead, I will include a category segment across treatment/control for repeat and new companies.
    - "Repeat" = companies that I've applied to before. I'll attempt to equally distribute these during assignment.
    - "New" = companies that I have not applied to before. I'll attempt to equally distribute these during assignment.

# Data Processing

In [1]:
_ = """
To-Do:
- (FIXED!) sortingHat: make it sort at stratification cluster level.


"""

In [1]:
# Import our python packages.
import os
import numpy as np
import pandas as pd
import random
from IPython.display import FileLink

#from sklearn.model_selection import StratifiedShuffleSplit <--this didn't work, because some groupings only have 1 job role.

#import the job sampling data
data_folder = "/notebooks/resume_experiment/data"
dataf = "modified_data.csv"
dataf2 = "modified_data II.csv"
data_file = os.path.join(data_folder, dataf)
data_file2 = os.path.join(data_folder,dataf2)

print(data_file)

/notebooks/resume_experiment/data/modified_data.csv


In [3]:
def procData(fname):
    data_folder = "/notebooks/resume_experiment/data"
    data_file = os.path.join(data_folder, fname)
    # Read in our job postings data from "modified_data.csv"
    raw_df = pd.read_csv(data_file, low_memory=False)
    raw_df = raw_df.dropna(axis=0,subset="role_title")
    raw_df = raw_df.loc[raw_df.loc[:,'DELETE Row'] != True]
    # Base the assignment on whether a repeat company and the type of role (data scientist, analyst, or manager/lead).
    raw_df['sorting_hat_col'] = raw_df['is_repeat_company'].astype('str')+"_"+raw_df['role_cat']
    raw_df = raw_df.drop(["Applied Jobs Check", "DELETE Row", "Check Unique Job/Co"], axis=1)
    return(raw_df)

In [8]:
# Read in our job postings data from "modified_data.csv"
#raw_df = pd.read_csv(data_file, low_memory=False)
#raw_df = raw_df.dropna(axis=0,subset="role_title")
#raw_df = raw_df.loc[raw_df.loc[:,'DELETE Row'] != True]
#
## Base the assignment on whether a repeat company and the type of role (data scientist, analyst, or manager/lead).
#raw_df['sorting_hat_col'] = raw_df['is_repeat_company'].astype('str')+"_"+raw_df['role_cat']
#raw_df = raw_df.drop(["Applied Jobs Check", "DELETE Row", "Check Unique Job/Co"], axis=1)
#
#display(raw_df.sample(5))
#display(raw_df.loc[raw_df.loc[:,"sorting_hat_col"].isnull()].head(10))

raw_df = procData(dataf)
raw_df2 = procData(dataf2)

In [5]:
#build a list of the companies that are in our dataset. include new vs. repeat as a dimension
companies = raw_df.copy().filter(['company','is_repeat_company','days_since_post'])
companies['counts'] = 1
agg_mets = {'counts':"count", 'days_since_post':"mean"}
companies = companies.groupby(['company','is_repeat_company']).agg(agg_mets).reset_index()
display(companies.head())

,company,is_repeat_company,counts,days_since_post
0,6sense,0.0,1,7.0
1,8x8,0.0,1,14.0
2,Abercrombie & Fitch Co.,0.0,1,14.0
3,Abnormal Security,0.0,1,14.0
4,Age of Learning,0.0,1,6.0


In [9]:
# Create a function to sort our job postings into two groups using a stratification column.
def sortingHat(data, stratification_col, split_percent=0.5, seed=42):
    """
    docstr by ChatGPT
    Stratify and assign job postings into two groups based on a given column.

    This function takes a dataset and a stratification column, then splits the rows 
    within each unique stratification group into two random groups (A/B splits) using 
    the provided split percentage and random seed.

    Args:
        data (pd.DataFrame): The input DataFrame containing job postings.
        stratification_col (str): The column name to use for stratification. Rows with 
            the same value in this column will be grouped and split together.
        split_percent (float, optional): The proportion of rows assigned to one of 
            the groups (default is 0.5, creating roughly equal splits).
        seed (int, optional): The random seed for reproducibility (default is 42).

    Returns:
        pd.DataFrame: A modified DataFrame with an additional column `'ab_split'`, 
        indicating group assignment (0 or 1) for each row.

    Raises:
        KeyError: If `stratification_col` is not a column in the input DataFrame.
        ValueError: If `split_percent` is not between 0 and 1.

    Examples:
        >>> import pandas as pd
        >>> import numpy as np
        >>> raw_df = pd.DataFrame({
        ...     "job_id": [1, 2, 3, 4, 5],
        ...     "department": ["HR", "Engineering", "HR", "Engineering", "HR"]
        ... })
        >>> sorted_df = sortingHat(raw_df, "department", split_percent=0.5)
        >>> sorted_df
           job_id     department  ab_split
        0       1             HR       0.0
        1       2  Engineering       1.0
        2       3             HR       1.0
        3       4  Engineering       0.0
        4       5             HR       0.0
    """
    # Ensure the stratification column exists in the DataFrame
    if stratification_col not in data.columns:
        raise KeyError(f"Column '{stratification_col}' not found in DataFrame.")

    # Validate split_percent
    if not (0 <= split_percent <= 1):
        raise ValueError("split_percent must be between 0 and 1.")
    
    #df = pd.DataFrame(columns=data.columns)
    #Create a list of unique stratification segments.
    uni_strats = data[stratification_col].unique()
    
    #Setting our seed
    rng = np.random.default_rng(seed)

    # Create a new column for the splits
    data['ab_split'] = np.nan
    
    for s in uni_strats:
       # Filter rows for the current stratification group
        sorting_df = data.loc[data[stratification_col] == s]
        nrows = sorting_df.shape[0]
        
        # Generate random splits and align indices
        split = pd.Series(rng.integers(low=0, high=1, size=nrows, endpoint=True), index=sorting_df.index)
        
        # Assign the split values to the DataFrame
        data.loc[sorting_df.index, 'ab_split'] = split
        
        #print(f"{s}: {sorting_df.shape}")
        #print(split)
        #display(data.loc[sorting_df.index].head(2))
        
    #uni_strats['ab_split'] = pd.Series(rng.integers(low=0, high=1, size=uni_strats.shape[0], endpoint=True), index=uni_strats.index)
    
    return(data)

sort_split = sortingHat(raw_df, 'sorting_hat_col', split_percent=0.5).sort_values(by="sorting_hat_col")
sort_split2 = sortingHat(raw_df2, 'sorting_hat_col', split_percent=0.5).sort_values(by="sorting_hat_col")
display(sort_split2)



    

,role_title,company,date_saved,posting_url,days_since_post,is_repeat_company,date_posted,job_posted_pin,role_cat,sorting_hat_col,ab_split
1,Senior Data Analyst (L3),Twilio,NaN,https://www.linkedin.com/jobs/view/4053088753/...,NaN,0.0,NaN,NaN,Data Analyst,0.0_Data Analyst,1.0
2,People Data Analytics Partner,Dropbox,NaN,https://www.linkedin.com/jobs/view/4077276401/...,NaN,0.0,NaN,NaN,Data Analyst,0.0_Data Analyst,0.0
25,"Senior Data Analyst, Fan Experience",Seat Geek,NaN,https://www.linkedin.com/jobs/view/4031366538/...,NaN,0.0,NaN,NaN,Data Analyst,0.0_Data Analyst,0.0
34,Senior Product Data Analyst,Customer.io,NaN,https://www.linkedin.com/jobs/view/4085340161/...,NaN,0.0,NaN,NaN,Data Analyst,0.0_Data Analyst,1.0
56,Senior Data Analyst,Brightwheel,NaN,https://www.linkedin.com/jobs/view/4050166663/...,NaN,0.0,NaN,NaN,Data Analyst,0.0_Data Analyst,0.0
...,...,...,...,...,...,...,...,...,...,...,...
59,Senior Data Scientist,Jerry,NaN,https://www.linkedin.com/jobs/search/?currentJ...,NaN,1.0,NaN,NaN,Data Scientist,1.0_Data Scientist,0.0
51,"Staff Data Scientist, Fraud",Angi,NaN,https://www.linkedin.com/jobs/view/4062600805/...,NaN,1.0,NaN,NaN,Data Scientist,1.0_Data Scientist,0.0
62,Sr Data Scientist,Microsoft,NaN,https://www.linkedin.com/jobs/search/?currentJ...,NaN,1.0,NaN,NaN,Data Scientist,1.0_Data Scientist,1.0
32,Data Scientist,Pinterest,NaN,https://www.linkedin.com/jobs/view/3965753694/...,NaN,1.0,NaN,NaN,Data Scientist,1.0_Data Scientist,1.0


In [6]:
#split_df = pd.merge(raw_df, sort_split, how='outer', on='sorting_hat_col').sort_values(by="sorting_hat_col")
#display(split_df)

In [10]:
split_path = os.path.join(data_folder,"split.csv")
sort_split.to_csv(split_path)

split_path2 = os.path.join(data_folder, "split2.csv")
sort_split2.to_csv(split_path2)
#print(len(grps))
#print(grps)

# Evaluate the Splits
Counting distribution of each split:
- Companies: applying again vs. never applied before.
- Role types: Analyst, Scientist, Manager
- Time since job post.

In [11]:
split_df = pd.read_csv(split_path,low_memory=False)
split_df2 = pd.read_csv(split_path2,low_memory=False)

In [15]:
def evalSplit(data, split_col):
    df = data.filter([split_col,'role_title','ab_split']).groupby([split_col, 'ab_split']).count()
    return(df)

def showEvalSplit(data):
    total_split = evalSplit(data, 'sorting_hat_col').reset_index()
    total_split = total_split.pivot(index="sorting_hat_col", columns="ab_split", values="role_title")
    display(total_split)

    company_split = evalSplit(data, 'is_repeat_company')
    display(company_split)

    role_type_split = evalSplit(data, 'role_cat')
    display(role_type_split)

    time_since_split = evalSplit(data, 'job_posted_pin')
    display(time_since_split)
    
showEvalSplit(split_df)
showEvalSplit(split_df2)

ab_split,0.0,1.0
sorting_hat_col,,
0.0_Data Analyst,27,24
0.0_Data Scientist,46,51
0.0_Manager,39,41
1.0_Data Analyst,7,3
1.0_Data Scientist,15,9
1.0_Manager,8,7


role_title
is_repeat_company ab_split            
0.0               0.0              112
                  1.0              116
1.0               0.0               30
                  1.0               19

role_title
role_cat       ab_split            
Data Analyst   0.0               34
               1.0               27
Data Scientist 0.0               61
               1.0               60
Manager        0.0               47
               1.0               48

role_title
job_posted_pin ab_split            
14days         0.0               40
               1.0               34
30days         0.0               19
               1.0               20
7days          0.0               79
               1.0               77
More30         0.0                4
               1.0                4

ab_split,0.0,1.0
sorting_hat_col,,
0.0_Data Analyst,5.0,6.0
0.0_Data Scientist,15.0,19.0
0.0_Manager,6.0,5.0
1.0_Data Analyst,2.0,4.0
1.0_Data Scientist,2.0,2.0
1.0_Manager,NaN,1.0


role_title
is_repeat_company ab_split            
0.0               0.0               26
                  1.0               30
1.0               0.0                4
                  1.0                7

role_title
role_cat       ab_split            
Data Analyst   0.0                7
               1.0               10
Data Scientist 0.0               17
               1.0               21
Manager        0.0                6
               1.0                6

,,role_title
job_posted_pin,ab_split,
